# Geometric Algebra Intro

## Motivation

Problems: 
* rotations = confusing
  * quaternions are 4D hyperspheres? wtf? where does the math come from?
  * rotations in 2D, 3D, 4D have completely different implementations
* handling of infinity in projective geometry = confusing  
  * Lots of if statements, special cases
* cross products = confusing  
  * Right hand rule?
* lie groups = confusing   

What we get:
* Naturally "typed" points, vectors, lines, planes
* Intuitive projections, intersections, metrics, norms on all types
* Exception/conditional free code
* Rotations and translations use the same formulas, on all types
* Formulas generalize to any number of dimensions
* Derive complex numbers, quaternions, dual quaternions, lie algebras

Used in more complex math. Quantum physics. Spacetime. Conformal geometry. Also much easier to explain to a child.

 is coordinate-free,  
• has a uniform representation for points, lines, and planes,  
• can calculate “parallel-safe” meet and join of these geometric entities,  
• provides compact expressions for all classical euclidean formulas and  
constructions, including distances and angles, perpendiculars and parallels,  
orthogonal projections, and other metric operations,  
• has a single, geometrically intuitive form for euclidean motions, one  
with a single representation for operators and operands,  
• provides automatic differentiation of functions of one or several variables,  
• provides a compact, efficient model for kinematics and rigid body mechanics,  
• lends itself to efficient, practical implementation, and  

## Algebraic System

We're going to build up a system for doing geometry using algebra. Algebra is the study of symbols. This system does not use matrices or vectors explicitly, instead defining symbols that follow certain algebraic rules, and using them to represent geometric concepts and operations on them.

We're going to start with standard case where $e_1, ..., e_N$ represent the bases of $\mathbb{R}^N$. These symbols get operated on just like any variables. Addition and multiplication behave as usual, but multiplication has a couple of special rules:

1) The square of a basis contracts to a scalar: $e_i e_i = 1$  
2) The product of two different bases is anti-commutative: $e_i e_j = -e_j e_i$

There are reasons these rules are defined, but generally we should use the simplest set of rules that allow us to build up abstractions and deal with complex topics, and these rules allow us to do a whole lot.

In [262]:
# Define R3 bases with sympy, which luckily supports non-commutative symbols.
import sympy as sm
e1, e2, e3 = sm.symbols('e1 e2 e3', commutative=False)

Define an expression that is analogous to the vector (1, 2, -3):

In [264]:
vec = e1 + 2 * e2 - 3 * e3
vec

e1 + 2*e2 - 3*e3

We've been taught we can't "multiply" two vectors, but there's nothing stopping an open-minded person from doing so.

In [265]:
expr = sm.expand((e1 + 2 * e2) * (e1 - 2 * e2))
expr

-2*e1*e2 + e1**2 + 2*e2*e1 - 4*e2**2

We don't yet know what this means, but multiplying elements like this is the main operation in geometric algebra.

## Simplification
So far we've only told sympy that the symbols do not commute (note how there are $e_2 e_1$ and $e_1 e_2$ terms, but we haven't applied our special simplifications. Let's create a function that does so.

In [326]:
def simp(a):
    """ Apply simplifications to bases until no changes occur. """
    subs = {
        # Rule 1) Square of a basis is one
        e1**2: 1,
        e2**2: 1,
        e3**2: 1,
        # Rule 2) Bases are anti-commutative, prefer increasing order
        e2*e1: -e1*e2,
        e3*e1: -e1*e3,
        e3*e2: -e2*e3
    }
    while True:
        b = sm.expand(a).subs(subs)
        if b == a:
            return b
        a = b

See that this behaves as expected on some simple examples:

In [327]:
simp(e1 * e1)  # Contracts

1

In [328]:
simp(e1 * e2)  # Nothing happens

e1*e2

In [329]:
simp(e2 * e1)  # Anti-commutes

-e1*e2

In [330]:
simp(e2 * e1 * e2)  # Anti-commutes then contracts

-e1

Now let's simplify our vector-vector product from before:

In [331]:
expr = sm.expand((e1 + 2 * e2) * (e1 - 2 * e2))
expr

-2*e1*e2 + e1**2 + 2*e2*e1 - 4*e2**2

In [332]:
simp(expr)

-3 - 4*e1*e2

Note that we are left with a scalar and what we'll call a *bivector*. This math easy to work through, all the squares contribute to the scalar component ($e_1^2 - 4e_2^2 = 1 - 4 = -3$), and anti-commutation combines the other terms ($-2 e_1 e_2 + 2 e_2 e_1 = -2 e_1 e_2 - 2 e_1 e_2 = -4 e_1 e_2$).

## Multivectors

In general we define a *$k$-vector* as the product of $k$ bases. For example:
 * $42$ is a 0-vector  
 * $3 e_1 + 2 e_2$ is a 1-vector  
 * $3 e_1 e_2 + 5 e_1 e_3$ is a 2-vector.
 
A general element of this algebra is called a *multivector*, which is a linear combination of $k$-vectors. In an algebra of dimension $N$, we can only have $k$-vectors up to $k=N$. Note how the simplifcation rules will cancel any higher order terms out:

In [333]:
simp(e1 * e2 * e3 * e2 * e2)

e1*e2*e3

In [334]:
simp(e1 * e2 * e3 * e2 * e2 * e1 * e2)

-e3

The highest order element we can have for $\mathbb{R}^3$ is the $3$-vector $e_1 e_2 e_3$. The general form of a multivector in this space is:

$c_1 + c_2 e_1 + c_3 e_2 + c_4 e_3 + c_5 e_1 e_2 + c_6 e_1 e_3 + c_7 e_2 e_3 + c_8 e_1 e_2 e_3$

In $\mathbb{R}^2$ we have only two bases and one bivector:

$c_1 + c_2 e_1 + c_3 e_2 + c_4 e_1 e_2$

Mathematically, this algebra is a ring - a set that defines addition and multiplication. It is also a closed group - any element of the algebra will have the above form for some coefficients $c$. It is commutative under addition but non-commutative under multiplication. It is always associative.

So far we've defined the geometric algebra for $\mathbb{R}^N$ but not interpreted or used it for geometry.

## Meaning
If $e_1$ and $e_2$ are unit vectors, what does the bivector $e_1 e_2$ represent? Intuitively, it represents an *oriented area*, in the same way that a vector represents an *oriented length*. It can be thought of as the plane defined by the two vectors, with an area spanned by their parallelogram:

<img style="float: left; width: 200px;" src="https://www.researchgate.net/profile/A_Arsenovic/publication/318475103/figure/fig6/AS:547679924756491@1507588477780/Geometric-algebra-for-three-dimensional-space-illustrating-the-vector-and-bivector-basis.png"/>

Similarly, a trivector represents an oriented volume. In $\mathbb{R}^3$ it's more like a signed volume since there is only one trivector $e_{123}$.

<img style="float: left; width: 250px;" src="https://i.stack.imgur.com/UNRIn.png"/>

The following is the general case for the product of two vectors in $\mathbb{R}^2$:

In [335]:
a, b, c, d = sm.symbols('a b c d')
simp((a * e1 + b * e2) * (c * e1 + d * e2))

a*c + a*d*e1*e2 - b*c*e1*e2 + b*d

Note how the product is a multivector that is the sum of a scalar $ac + bd$ and a bivector $(ad - bc) e_1 e_2$. How do we interpret this? Well, they are both interesting quantities but we generally treat them separately. The 

grade operation

$a = (ac + bd) + (ad - bc) e_1 e_2$  
$\langle a \rangle_0 = ac + bd$  
$\langle a \rangle_1 = 0$  
$\langle a \rangle_2 = (ad - bc) e_1 e_2$  
$\langle a \rangle_3 = 0$  

inner product, outer product

In [336]:
simp((e1 + 2*e2 + 3*e3)**2)

14

## Homogenous (projective) space
cayley tables

## Motors

exponential

## Lie Algebra

foiefwe